# 📓 The GenAI Revolution Cookbook

**Title:** How to Build a Model Context Protocol (MCP) Server in Python

**Description:** Learn how to build an MCP server in Python to standardize and reuse AI tools, resources, and prompts across applications. This hands-on guide walks you through server setup, client testing, and GPT-4 chatbot integration for production-ready systems.

**📖 Read the full article:** [How to Build a Model Context Protocol (MCP) Server in Python](https://blog.thegenairevolution.com/article/how-to-build-a-model-context-protocol-mcp-server-in-python)

---

*This jupyter notebook contains executable code examples. Run the cells below to try out the code yourself!*



The Model Context Protocol (MCP) lets you define tools, resources, and prompts once and expose them to any MCP\-capable client. This includes CLIs, agents, and chatbots. Instead of rewriting tool logic for each application, you build a single server that clients discover and invoke automatically.

This guide walks you through building a minimal Python MCP server over stdio. You'll test it with a client and verify that tools, resources, and prompts work as expected. By the end, you'll have a working server that exposes math tools, static documentation, and a prompt template. You can run it in a notebook or a local terminal.

For a comprehensive overview of how MCP standardizes tool and data access, see [Model Context Protocol (MCP) Explained.](/article/model-context-protocol-mcp-explained-2025-guide-for-builders)

## Why Use MCP for This Problem

When you need to share tools across multiple applications, you have several options.

* **Shared Python package**: Every app must import and maintain the same library version. You get no runtime discovery or schema negotiation.

* **Bespoke HTTP microservice**: You add network overhead and custom API design. You also lack standardized tool metadata.

* **OpenAI\-native tools defined per app**: You duplicate tool definitions in every codebase. You lose a single source of truth.

* **Agent\-framework\-specific tools**: You get locked into one framework API. Porting to another requires rewriting.

MCP solves these problems by providing:

* **Automatic discovery**: Clients list available tools, resources, and prompts at runtime.

* **Standardized schemas**: JSON Schema definitions ensure consistent validation and documentation.

* **Transport abstraction**: Stdio, SSE, or WebSocket. Clients and servers negotiate capabilities without custom protocols.

Centralizing these capabilities in one server removes repetition. Clients can discover and invoke standardized functionality automatically. If you want to avoid subtle bugs caused by tokenization quirks, read [Tokenization Pitfalls: Invisible Characters That Break Prompts and RAG](/article/tokenization-pitfalls-invisible-characters-that-break-prompts-and-rag-2), and consider [implementing reliable vector store retrieval for RAG](/article/rag-101-build-an-index-run-semantic-search-and-use-langchain-to-automate-it).

A single MCP server gives you one place to update logic and metadata. You get one schema for validation and consistent behavior across apps. Keep in mind that LLM context is not infinite memory. If you plan to scale prompt sizes or chain calls, see [Context Rot \- Why LLMs "Forget" as Their Memory Grows](/article/context-rot-why-llms-forget-as-their-memory-grows-3).

If you want to compare MCP with a workflow\-first approach, explore [building robust LLM workflows with LangChain](/article/langchain-101-build-your-first-real-llm-application-step-by-step). It complements this pattern by showing how to orchestrate tools and prompts in a structured pipeline.

## Core Concepts for This Use Case

Before building, let's understand these MCP primitives.

* **Tools**: Functions the client can invoke with typed arguments. The server executes and returns results.

* **Resources**: Static or dynamic data identified by URI. Clients read them on demand as text, JSON, or binary.

* **Prompts**: Reusable templates that generate messages for LLM conversations. Clients fetch and render them with arguments.

* **Stdio transport**: Server and client communicate over standard input and output. This is the simplest option for local development and subprocess integration.

* **Schema negotiation**: The server advertises tool input schemas with JSON Schema and resource MIME types. Clients validate input and adapt automatically.

If you're new to crafting effective templates and roles, see [best practices for prompt engineering with LLM APIs](/article/prompt-engineering-with-llm-apis-how-to-get-reliable-outputs-4). It will help you get consistent results from your MCP prompts.

## Setup

Confirm you have Python 3\.9 or later. Create a clean virtual environment to avoid dependency conflicts. Decide whether you'll run in a notebook or in a terminal. If you use a notebook, make sure you can write files to the working directory.

Install the required packages in a notebook cell or terminal:

In [ ]:
!pip install "mcp[cli]>=0.9.0" anyio>=4.0.0 openai>=1.40.0

This installs the MCP SDK, an async runtime, and an OpenAI client. Pin versions for reproducibility. If installation fails, check your Python version and virtual environment first. Then retry the install. If you plan to operate your stack outside managed services, consider [deploying a self\-hosted LLM server](/article/how-to-run-a-self-hosted-llm-on-your-server-practical-guide-2025-2) to pair with your MCP server.

## Using the Tool in Practice

### Build the MCP Server

Create a server that exposes two math tools, a static markdown resource, and a prompt template. This example uses stdio transport for simplicity.

Decide on a filename for the server. For example, you can name it mcp\_server.py. If you work in a notebook, use your environment's file writing utility to create the file. If you work in a terminal, create the file with your preferred editor.

Write the server to a file using a notebook cell:

In [ ]:
%%writefile mcp_server.py
# Purpose: Minimal MCP server exposing math tools, a static resource, and a prompt template over stdio.

import anyio
from typing import Annotated

# MCP server APIs
from mcp.server import Server
from mcp.server.stdio import stdio_server
from mcp.types import (
    PromptMessage,
    TextResourceContents,
    Resource,
    Prompt,
)

# Instantiate the MCP server with a unique name
server = Server("calc-server")

@server.tool()
async def add(a: Annotated[int, "First integer"], b: Annotated[int, "Second integer"]) -> int:
    """
    Add two integers and return the sum.

    Args:
        a (int): First integer.
        b (int): Second integer.

    Returns:
        int: The sum of a and b.
    """
    # Simple addition; no edge cases for int
    return a + b

@server.tool()
async def subtract(a: Annotated[int, "Minuend"], b: Annotated[int, "Subtrahend"]) -> int:
    """
    Subtract b from a and return the difference.

    Args:
        a (int): Minuend.
        b (int): Subtrahend.

    Returns:
        int: The result of a - b.
    """
    # Simple subtraction; no edge cases for int
    return a - b

# Resource: static markdown documentation
DOCS_ID = "docs/usage"
DOCS_CONTENT = """# Calc Server Usage

Tools:
- add(a: int, b: int): returns a + b
- subtract(a: int, b: int): returns a - b

Prompt:
- math_helper(expression: string): step-by-step computation
"""

@server.resource(DOCS_ID, mime_type="text/markdown")
async def read_docs() -> TextResourceContents:
    """
    Return static markdown documentation for the server.

    Returns:
        TextResourceContents: Markdown-formatted usage documentation.
    """
    # TextResourceContents includes text and optional annotations
    return TextResourceContents(text=DOCS_CONTENT)

@server.prompt("math_helper")
async def math_helper(expression: Annotated[str, "Math expression to compute"]):
    """
    Generate a prompt for step-by-step math computation.

    Args:
        expression (str): Math expression to compute.

    Returns:
        list[PromptMessage]: System and user prompt messages.
    """
    # System prompt instructs the assistant to show work
    system = PromptMessage(role="system", content="You are a careful math assistant. Show your work.")
    # User prompt includes the expression to compute
    user = PromptMessage(role="user", content=f"Compute the following expression step by step: {expression}")
    return [system, user]

async def main():
    """
    Main entry point: runs the MCP server over stdio until EOF.
    """
    # stdio transport: run until EOF
    async with stdio_server() as (read_stream, write_stream):
        await server.run(read_stream, write_stream)

if __name__ == "__main__":
    anyio.run(main)

The @server.tool() decorator registers async functions as tools. The client receives JSON Schema for each tool, so it can validate arguments before calling. The @server.resource() decorator exposes static or dynamic data by URI. Clients read the content using the resource URI, and the server returns the body with a MIME type. The @server.prompt() decorator defines reusable prompt templates. A prompt can define named input variables, message roles, and a message sequence. The server runs over stdio, reads requests from stdin, and writes responses to stdout.

As you build, keep these checks in mind:

* Each tool should define a clear JSON Schema for inputs. This helps clients validate before execution.

* Each resource should return a stable URI and a correct content type. This helps clients render content correctly.

* Each prompt should declare required arguments and message roles. This helps clients supply all needed variables.

### Test the Server with a Client

Create a separate file for the client. You can name it mcp\_client.py or reuse a notebook cell. The client will start the server as a subprocess and connect over stdio.

Write a client that connects to the server, lists capabilities, calls a tool, reads a resource, and fetches a prompt:

In [ ]:
%%writefile mcp_client.py
# Purpose: Async MCP client to test server tools, resources, and prompts over stdio.

import anyio
from mcp.client.stdio import stdio_client
from mcp.client.session import ClientSession

async def main():
    """
    Connects to the MCP server, lists tools/resources/prompts, calls a tool, and fetches a prompt.

    Raises:
        Exception: If server connection or calls fail.
    """
    # Launch the server as a subprocess and connect over stdio
    async with stdio_client(["python", "mcp_server.py"]) as (read_stream, write_stream):
        async with ClientSession(read_stream, write_stream) as session:
            # List tools and print their names
            tools = await session.list_tools()
            print("Tools:", [t.name for t in tools.tools])

            # List resources and print their URIs
            resources = await session.list_resources()
            print("Resources:", [r.uri for r in resources.resources])

            # Read and print documentation resource if present
            for r in resources.resources:
                if r.uri.endswith("docs/usage"):
                    content = await session.read_resource(r.uri)
                    # content.contents is a list of typed chunks (e.g., text, blob)
                    for c in content.contents:
                        if hasattr(c, "text"):
                            print("Docs:\n", c.text)

            # Call 'add' tool with arguments and print the result
            result = await session.call_tool("add", {"a": 5, "b": 7})
            # result.content is list of output messages; pick first text
            out = result.content[0].text if result.content else None
            print("add(5,7) =", out)

            # Fetch prompt template and print its messages
            prompts = await session.list_prompts()
            print("Prompts:", [p.name for p in prompts.prompts])
            prompt = await session.get_prompt("math_helper", {"expression": "3*(4+2)"})
            print("Prompt messages:", [(m.role, m.content) for m in prompt.messages])

if __name__ == "__main__":
    anyio.run(main)

The client launches the server as a subprocess, establishes a session over stdio, and exercises all three primitives. It lists tools, resources, and prompts. It calls a math tool to confirm schema validation and execution. It reads a markdown resource to confirm URI resolution and content types. It fetches a prompt and renders it with arguments. The stdio\_client context manager handles process lifecycle and stream wiring. If the server crashes, you'll see an error during capability discovery or the first call. Check the server logs or print statements to diagnose.

### Run and Evaluate

Run the client to verify the server works end to end. If you use a terminal, run the client from the same virtual environment. If you use a notebook, run the cell that executes the client code.

In [ ]:
!python mcp_client.py

Expected output:

In [ ]:
Tools: ['add', 'subtract']
Resources: ['docs/usage']
Docs:
 # Calc Server Usage

Tools:
- add(a: int, b: int): returns a + b
- subtract(a: int, b: int): returns a - b

Prompt:
- math_helper(expression: string): step-by-step computation

add(5,7) = 12
Prompts: ['math_helper']
Prompt messages: [('system', 'You are a careful math assistant. Show your work.'), ('user', 'Compute the following expression step by step: 3*(4+2)')]

This output confirms that:

* The client discovered tools, resources, and prompts during session setup.

* The tool call returned a valid result and matched the expected schema.

* The resource read returned the expected content with the correct MIME type.

* The prompt rendered correctly with your input arguments.

If any step fails, verify your Python path, virtual environment, and file locations. Make sure your server file is executable in your environment. On Windows, confirm that your Python launcher runs the correct interpreter. On macOS or Linux, confirm permissions and the working directory.

### Optional: Add Resilience to Tool Calls

For production use, add a timeout and error handler around tool calls. This prevents a slow or failing tool from blocking the client. You'll also capture and surface meaningful error messages.

Create a helper function:

In [ ]:
%%writefile helpers/safe_call.py
# Purpose: Utility for safe, timeout-guarded MCP tool calls.

from anyio import fail_after, WouldBlock
import logging

async def safe_call_tool(mcp, name, args, seconds=10):
    """
    Call an MCP tool with a timeout and error handling.

    Args:
        mcp: MCP client session.
        name (str): Tool name.
        args (dict): Tool arguments.
        seconds (int): Timeout in seconds.

    Returns:
        Tool call result or None if failed/timed out.

    Raises:
        None: All exceptions are caught and logged.
    """
    try:
        with fail_after(seconds):
            return await mcp.call_tool(name, args)
    except WouldBlock:
        # Timeout occurred
        logging.warning(f"Tool call to {name} timed out after {seconds}s.")
        return None
    except Exception as e:
        # Log error and return None
        logging.error(f"Tool call {name} failed: {e}")
        return None

Use safe\_call\_tool in place of direct session.call\_tool. Start with a conservative timeout. Expand it only if you confirm that a tool needs more time. Always log errors and return structured error information to the caller.

## Troubleshooting

* **The client cannot discover tools**: Ensure the server process starts correctly. Check that the stdio transport is not buffered or redirected. Confirm the working directory contains the server file.

* **JSON Schema validation errors**: Verify that your tool input matches the schema. Check required fields, types, and enum values. Update the tool schema if your arguments are valid but the schema is too strict.

* **Resource not found**: Confirm the resource URI matches exactly. If the server builds resource URIs dynamically, print the final URI and compare it to the client request.

* **Prompt rendering errors**: Make sure you pass all required variables. If your template variables change, update both the server prompt declaration and the client call.

* **Hanging calls**: Add timeouts using the resilience pattern. If a tool depends on external services, handle network retries and backoff inside the tool implementation.

* **Version conflicts**: Pin your dependencies and reinstall in a fresh virtual environment. Confirm that the client and server rely on compatible MCP SDK versions.

## Conclusion

You built a minimal MCP server that exposes tools, resources, and prompts over stdio, and you verified it with a test client. This pattern lets you define capabilities once and reuse them across any MCP\-compatible application. You now have a repeatable way to standardize tool access without rewriting code for each app.

## Next Steps

* **Integrate with OpenAI function calling**. Convert MCP tools to OpenAI tool schemas and route calls from GPT\-4 to your server. This is covered in a separate guide. For deeper customization, see the [step\-by\-step guide to fine\-tuning large language models](/article/fine-tuning-large-language-models-a-step-by-step-guide-2025-6).

* **Add dynamic resources**: Serve real\-time data like database queries or API responses instead of static markdown.

* **Explore other transports**: Use SSE or WebSocket for remote or browser\-based clients.

* **Package and deploy**: Containerize your server and expose it through a network transport for multi\-user access.